#SLEAP_AI.ipnyb

#Description
---------
This notebook runs SLEAP.ai in the colab Virtual Machine.

It uses the power of NVIDIA Tesla T4 GPU to take the best out of SLEAP running it's deep learning neural networks faster.

The system used is condacolab since it helps bypass default python limitations from colab

SLEAP is run inside an environment where it has the necessary drivers.

Run order

1.   Choose the Paths.
2.   Prepare the environment
3.   Run SLEAP
4.   Output the predictions



SLEAP.ai DOI 🔗 https://doi.org/10.1038/s41592-022-01426-1

Moita Lab 🔗 https://moitalab.org/




#Define Paths

In [1]:
import os
from IPython.display import display, clear_output
import ipywidgets as widgets
from google.colab import drive
from ipyfilechooser import FileChooser

# 1. Mount Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Repository Settings (Fixed Model)
REPO_URL = "https://github.com/moitalab/Sleap_Colab.git"
REPO_DIR = "/content/Sleap_Colab"

# Automatic repository download (including best_model.h5 via LFS)
if not os.path.exists(REPO_DIR):
    print("🚀 Downloading model and scripts from GitHub (Moita Lab)...")
    !git clone {REPO_URL}
else:
    print("✅ GitHub model is already prepared.")

# --- SELECTION INTERFACE ---
chooser_exp = FileChooser('/content/drive/MyDrive/')
chooser_exp.title = '<b>Select Experiment Folder (Root)</b>'
chooser_exp.show_only_dirs = True

save_button = widgets.Button(
    description='Confirm Configuration',
    button_style='success',
    icon='check',
    layout=widgets.Layout(width='300px')
)

output_log = widgets.Output()

def on_confirm_clicked(b):
    with output_log:
        clear_output()
        exp_root = chooser_exp.selected_path
        if not exp_root:
            print("❌ Error: Please select the experiment folder in Drive!")
            return

        # Standard repository subfolder paths
        crop_raw = os.path.join(exp_root, "PostProcessing", "CropRaw")
        arenas = os.path.join(exp_root, "PostProcessing", "Arenas")
        pose_folder = os.path.join(exp_root, "PostProcessing", "Pose")

        # Ensure the output directory exists
        os.makedirs(pose_folder, exist_ok=True)

        # Save paths for the Bash environment
        with open('/content/sleap_paths.env', 'w') as f:
            f.write(f'export VIDEO_FOLDER="{crop_raw}"\n')
            f.write(f'export ARENAS_FOLDER="{arenas}"\n')
            f.write(f'export OUTPUT_FOLDER="{pose_folder}"\n')
            f.write(f'export MODEL_PATH="{REPO_DIR}"\n')

        print("-" * 50)
        print("✅ PIPELINE CONFIGURED!")
        print(f"🧠 Model: Loaded from GitHub")
        print(f"📍 Videos: {crop_raw}")
        print("-" * 50)

save_button.on_click(on_confirm_clicked)
display(chooser_exp, save_button, output_log)

Mounted at /content/drive
🚀 A descarregar modelo e scripts do GitHub (Moita Lab)...
Cloning into 'Sleap_Colab'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 1), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 13.90 KiB | 273.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


FileChooser(path='/content/drive/MyDrive', filename='', title='<b>Selecionar Pasta da Experiência (Root)</b>',…

Button(button_style='success', description='Confirmar Configuração', icon='check', layout=Layout(width='300px'…

Output()

#Enviroment preparation

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install() # Runtime will restart

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


In [3]:
!mamba create -y -n sleap_env -c conda-forge -c nvidia -c sleap -c anaconda \
    python=3.7 \
    sleap=1.3.4 \
    tensorflow=2.7.0 \
    cudatoolkit=11.3.1 \
    cudnn=8.2.1.32 \
    pandas=1.3.5 \
    scipy=1.7.3 \
    matplotlib=3.5.3

A saída de streaming foi truncada nas últimas 5000 linhas.
Transaction

  Prefix: /usr/local/envs/sleap_env

  Updating specs:

   - python=3.7
   - sleap=1.3.4
   - tensorflow=2.7.0
   - cudatoolkit=11.3.1
   - cudnn=8.2.1.32
   - pandas=1.3.5
   - scipy=1.7.3
   - matplotlib=3.5.3


  Package                    Version  Build                   Channel           Size
──────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────

  + cuda-nvcc                11.3.58  h2467b9f_0              nvidia            57MB
  + python_abi                   3.7  4_cp37m                 conda-forge        6kB
  + ld_impl_linux-64          2.45.1  bootstrap_ha15bf96_1    conda-forge      726kB
  + libgomp                   15.2.0  he0feb66_18             conda-forge      603kB
  + _openmp_mutex                4.5  20_gnu                  conda-forge       29kB
  + libgcc        

In [4]:
%%bash

# Test if th gpu is available
source activate sleap_env
python -c "import sleap; print('SLEAP Version Check:', sleap.__version__)"
python -c "import tensorflow as tf; print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))"

INFO:matplotlib.font_manager:generated new fontManager
SLEAP Version Check: 1.3.4
Num GPUs Available:  1


2026-02-19 16:13:34.554973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-19 16:13:34.686742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-19 16:13:34.689660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


#Running


In [1]:
%%bash
# 1. Load variables
source /content/sleap_paths.env
source activate sleap_env
export MPLBACKEND=Agg

# --- FIX: Clean absolute paths in JSON ---
python -c "
import json
import os
config_path = os.path.join(os.environ['MODEL_PATH'], 'training_config.json')
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)

    # Remove fixed paths (e.g., H:\...) so SLEAP uses the local .h5 file
    heads = config.get('model', {}).get('heads', {})
    for head_name in heads:
        if isinstance(heads[head_name], dict):
            for key in list(heads[head_name].keys()):
                if 'keras_model_path' in key:
                    heads[head_name][key] = None

    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)
    print('✅ Model configuration optimized for Colab.')
"

# 2. Batch Processing
for video in "$VIDEO_FOLDER"/*.avi; do
    [ -e "$video" ] || continue
    filename=$(basename "$video")
    current_basename="${filename%_crop.avi}"

    echo "------------------------------------------------"
    echo "PROCESSING: $filename"

    # STEP 1: SLEAP Inference
    sleap-track "$video" \
        -m "$MODEL_PATH/training_config.json" \
        --output "$OUTPUT_FOLDER/${filename}.predictions.slp" \
        --no-gui

    # STEP 2: Normalization and CSV Generation (Python)
    export CUR_FILE="$filename"
    export CUR_BASE="$current_basename"

    python - <<EOF
import sleap
import pandas as pd
import os
import numpy as np
from PIL import Image

arenas_path = os.environ.get('ARENAS_FOLDER', '')
output_path = os.environ.get('OUTPUT_FOLDER', '')
video_file = os.environ.get('CUR_FILE', '')
basename = os.environ.get('CUR_BASE', '')

slp_file = os.path.join(output_path, f"{video_file}.predictions.slp")
csv_output = os.path.join(output_path, f"{basename}_pose.csv")
png_file = os.path.join(arenas_path, f"{basename}.png")

node_mapping = {'L': 'Left', 'R': 'Right', 'H': 'Head', 'Trx': 'Thorax',
                'Abd': 'Abdomen', 'Lw': 'LeftWing', 'Rw': 'RightWing', 'T': 'Top'}

if os.path.exists(slp_file):
    w, h = 1.0, 1.0
    if os.path.exists(png_file):
        with Image.open(png_file) as img:
            w, h = img.size
        print(f"-> Arena Resolution: {w}x{h}")

    labels = sleap.load_file(slp_file)
    data = []
    for frame in labels:
        for inst in frame.instances:
            row = {'FrameIndex': frame.frame_idx}
            for node in labels.skeleton.nodes:
                b_name = node_mapping.get(node.name, node.name)
                pt = inst[node.name]
                if pt is not None:
                    row[f'{b_name}.Position.X'] = pt.x / w
                    row[f'{b_name}.Position.Y'] = pt.y / h
                    row[f'{b_name}.Confidence'] = pt.score
                else:
                    row[f'{b_name}.Position.X'] = np.nan
                    row[f'{b_name}.Position.Y'] = np.nan
                    row[f'{b_name}.Confidence'] = 0.0
            data.append(row)

    df = pd.DataFrame(data)
    if not df.empty:
        df.sort_values('FrameIndex').to_csv(csv_output, index=False, na_rep='NaN')
        print(f"✅ Success: {basename}_pose.csv generated.")
EOF
done
echo "================================================"
echo "PROCESS COMPLETE!"

✅ Configuração do modelo otimizada para o Colab.
------------------------------------------------
A PROCESSAR: MF-LC6_X_CS_ChS(Retinal)-1Loom_19Opto(15mWcm2)-Female-4days-FH2-CamA-2025-08-15T14_30_05_fly0_crop.avi
Started inference at: 2026-02-19 16:13:49.477861
Args:
{
│   'data_path': '/content/drive/.shortcut-targets-by-id/1EvQfnCVnY3B5N4bSdunnJEZzZHMVxaoS/Matheus_e_Rodrigo/colab_bonsai/PostProcessing/CropRaw/MF-LC6_X_CS_ChS(Retinal)-1Loom_19Opto(15mWcm2)-Female-4days-FH2-CamA-2025-08-15T14_30_05_fly0_crop.avi',
│   'models': ['/content/Sleap_Colab/training_config.json'],
│   'frames': '',
│   'only_labeled_frames': False,
│   'only_suggested_frames': False,
│   'output': '/content/drive/.shortcut-targets-by-id/1EvQfnCVnY3B5N4bSdunnJEZzZHMVxaoS/Matheus_e_Rodrigo/colab_bonsai/PostProcessing/Pose/MF-LC6_X_CS_ChS(Retinal)-1Loom_19Opto(15mWcm2)-Female-4days-FH2-CamA-2025-08-15T14_30_05_fly0_crop.avi.predictions.slp',
│   'no_empty_frames': False,
│   'verbosity': 'rich',
│   'video.data

2026-02-19 16:13:49.608235: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-19 16:13:49.621458: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-19 16:13:49.624032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-19 16:13:49.783061: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil